In [1]:
%cd "/content/drive/My Drive/Colab_Notebooks/ghana_pop"

/content/drive/My Drive/Colab_Notebooks/ghana_pop


In [0]:
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd 
import os
import re

In [0]:
base_url = "https://www.statsghana.gov.gh/"

In [0]:
r = requests.get(base_url)
c = r.content
soup = BS(c, "html.parser")
# check source code of loaded page

# # extract regional links
all = soup.find_all("g") #.get('href')
all

In [0]:
region_links = []
for x in all:
  # extract link and clean
  link = x.find('a').get('href').replace("®","&reg")
  region_links.append(link)
  # print(x.find('a').get('href'))

In [0]:
# collect page contents for each region:
for link in region_links:
  url = base_url + link
  print (url)

In [0]:
reg_2 = "https://www.statsghana.gov.gh/regionalpopulation.php?population=MTMxOTU3MTAxOC44MjU=&&Bono&regid=2"

In [0]:
r = requests.get(reg_2)
c = r.content
soup = BS(c, "html.parser")
# check source code of loaded page
# # extract regional links

region_name = soup.find('div',{'class' :'graphWrap'}).find('h2').text #.get('href')
# len(all)

In [0]:
# grab table and details
reg_2table = pd.read_html(reg_2, attrs = {'class':'TDownloadDoc'})

In [0]:
K = reg_2table[0][('Year', 'District Name')]
V = reg_2table[0][('Population',    'Population')]


In [0]:
reG = pd.DataFrame(zip(K, V)).dropna()
reG.columns = ['district','population']
# clean up
reG['district'][0] = region_name + str(' REGION')
reG.drop(2,axis=0)
reG



In [0]:
for reg_url in extract_links(all):
    region_name = gather_region_name(reg_url)
    # return reg_table
    region_data = pd.read_html(reg_url, attrs = {'class':'TDownloadDoc'})
    K = region_data[0][('Year', 'District Name')]
    V = region_data[0][('Population',    'Population')]
    reg_df = pd.DataFrame(zip(K, V)).dropna()
    reg_df.columns = ['district','population']
    reg_df['district'][0] = region_name + str(' REGION')
    # # clean up
    reg_df = reg_df.drop(2,axis=0)
    # print(reg_df)
    GH_pop = pd.concat([GH_pop,reg_df],ignore_index=True)

In [0]:
def gather_page_all(url,element):
  r = requests.get(url)
  c = r.content
  soup = BS(c, "html.parser")
# check source code of loaded page
  all = soup.find_all(element) #.get('href')
  return all

def gather_region_name(reg_url):
  r = requests.get(reg_url)
  c = r.content
  soup = BS(c, "html.parser")
# check source code of loaded page
  region_name = soup.find('div',{'class' :'graphWrap'}).find('h2').text
  return region_name


In [0]:
def extract_links(all):
  region_links = []
  for item in all:
    # extract link and clean
    href = item.find('a').get('href').replace("®","&reg")
    link = base_url + href
    # print (link)
    region_links.append(link)
  return region_links 
    

In [0]:
def collect_page_details():
  GH_pop = pd.DataFrame()
  for reg_url in extract_links(all):
      region_name = gather_region_name(reg_url)
      # return reg_table
      region_data = pd.read_html(reg_url, attrs = {'class':'TDownloadDoc'})
      K = region_data[0][('Year', 'District Name')]
      V = region_data[0][('Population',    'Population')]
      reg_df = pd.DataFrame(zip(K, V)).dropna()
      reg_df.columns = ['district','population']
      reg_df['district'][0] = region_name + str(' REGION')
      # # clean up
      reg_df = reg_df.drop(2,axis=0)
      # print(reg_df)
      GH_pop = pd.concat([GH_pop,reg_df],ignore_index=True)
  GH_pop.to_csv('gh_pop.csv')
  return GH_pop

In [0]:
def main():
  gather_page_all(base_url,"g")
  extract_links(all)
  collect_page_details()

if __name__ == '__main__':
  base_url = "https://www.statsghana.gov.gh/"
  main()
  
